IMPORT

In [43]:
import requests
import json
import pandas as pd
import mysql.connector
import sys
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

CONNECTING TO DATABASE

In [44]:
mydb = mysql.connector.connect(
host="localhost",
user="admin",
password="admin",
database='cricinfo',
)
mycursor = mydb.cursor()

INPUT

In [45]:
team1_id = int(input("Enter the first team ID: "))
team1 = []
for i in range(1):
    player = int(input("Enter the player ID of Team 1: "))
    team1.append(player)

team2_id = int(input("Enter the second team ID: "))
team2 = []
for i in range(1):
    player = int(input("Enter the player ID of Team 2: "))
    team2.append(player)

match_type = int(input("Enter the match type: "))

ground_id = int(input("Enter the ground ID: "))

batting_first = int(input("Enter the batting first team (1 or 2): "))
bowling_first = int(input("Enter the bowling first team (1 or 2): "))
if batting_first == 1:
    team1_innings = 1
    team2_innings = 2
else:
    team1_innings = 2
    team2_innings = 1

ValueError: invalid literal for int() with base 10: ''

FETCHING REQUIRED DATA

In [46]:
# Fetching data for all 22 players
player_data_all_format = []
player_data_match_format = []
player_data_ground = []
player_data_opponent = []
player_data_innings = []
player_data_all = []

# Fetching data based on all formats
team = [379140]
for player_id in team1 + team2:
    query = """SELECT SUM(Total_Fantasy) FROM fantasy_point_table WHERE Player_ID = %s"""
    mycursor.execute(query,(player_id,))
    result = mycursor.fetchone()
    player_data_all_format.append(result[0])

# Fetching data based on match format
for player_id in team1 + team2:
    query = """SELECT SUM(Total_Fantasy) FROM fantasy_point_table WHERE Player_ID = %s AND Match_Type = %s"""
    mycursor.execute(query,(player_id,match_type))
    result = mycursor.fetchone()
    player_data_match_format.append(result[0])

# Fetching data based on ground
for player_id in team1 + team2:
    query = """SELECT SUM(Total_Fantasy) FROM fantasy_point_table WHERE Player_ID = %s AND Ground_ID = %s"""
    mycursor.execute(query,(player_id,ground_id))
    result = mycursor.fetchone()
    player_data_ground.append(result[0])

# Fetching data based on opponent
for player_id in team1:
    query = """SELECT SUM(Total_Fantasy) FROM fantasy_point_table WHERE Player_ID = %s AND Opponent_Team_ID = %s"""
    mycursor.execute(query,(player_id,team2_id))
    result = mycursor.fetchone()
    player_data_opponent.append(result[0])

for player_id in team2:
    query = """SELECT SUM(Total_Fantasy) FROM fantasy_point_table WHERE Player_ID = %s AND Opponent_Team_ID = %s"""
    mycursor.execute(query,(player_id,team1_id))
    result = mycursor.fetchone()
    player_data_opponent.append(result[0])

# Fetching data based on innings
for player_id in team1:
    query = """SELECT SUM(Total_Fantasy) FROM fantasy_point_table WHERE Player_ID = %s AND Innings_Order = %s"""
    mycursor.execute(query,(player_id,team1_innings))
    result = mycursor.fetchone()
    player_data_innings.append(result[0])

for player_id in team2:
    query = """SELECT SUM(Total_Fantasy) FROM fantasy_point_table WHERE Player_ID = %s AND Innings_Order = %s"""
    mycursor.execute(query,(player_id,team2_innings))
    result = mycursor.fetchone()
    player_data_innings.append(result[0])

# Fetching data based on match format, ground, opponent, and innings combined
for player_id in team1:
    query = """SELECT SUM(Total_Fantasy) FROM fantasy_point_table WHERE Player_ID = %s AND Match_Type = %s AND Ground_ID = %s AND Opponent_Team_ID = %s AND Innings_Order = %s"""
    mycursor.execute(query,(player_id,match_type,ground_id,team2_id,team1_innings))
    result = mycursor.fetchone()
    player_data_all.append(result[0])

for player_id in team2:
    query = """SELECT SUM(Total_Fantasy) FROM fantasy_point_table WHERE Player_ID = %s AND Match_Type = %s AND Ground_ID = %s AND Opponent_Team_ID = %s AND Innings_Order = %s"""
    mycursor.execute(query,(player_id,match_type,ground_id,team1_id,team2_innings))
    result = mycursor.fetchone()
    player_data_all.append(result[0])

In [47]:
print(player_data_all_format)
print(player_data_match_format)
print(player_data_ground)
print(player_data_opponent)
print(player_data_innings)
print(player_data_all)


[Decimal('13926'), Decimal('12009')]
[Decimal('9447'), Decimal('1700')]
[Decimal('118'), Decimal('280')]
[Decimal('1611'), Decimal('524')]
[Decimal('7367'), Decimal('5375')]
[Decimal('91'), Decimal('201')]


Data Set

In [48]:
query = "SELECT * FROM fantasy_point_table"
mycursor.execute(query)
rows = mycursor.fetchall()

# Convert the data into a pandas DataFrame
df = pd.DataFrame(rows, columns=[i[0] for i in mycursor.description])

In [ ]:
df.to_excel('filename.xlsx', index=False)

In [49]:
X = df.drop(['In_Dream_Team','Match_Date','Match_ID','Match_Player_ID','Captain','Vice_Captain','Status','Player_Name','Matches','Total_Fantasy','Batting_Fantasy','Bowling_Fantasy','Fielding_Fantasy','Bat_Innings','Bowl_Innings','Player_Rank','Value_Fantasy'], axis=1)
y = df['In_Dream_Team']

In [50]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train)

        Player_ID  Team_ID  Opponent_Team_ID  Ground_ID  Innings_Order  \
297817    1283583      122                35      58679              1   
128697    1087882  1174604              3570      59294              1   
67195      379239     3278            256841      59148              1   
36717      851261     2012              1668      58040              1   
146806     316363  1117814            953835      59396              1   
...           ...      ...               ...        ...            ...   
259178     820691   953845            953833      58956              1   
365838     233514   325788            325787      59306              2   
131932    1122918   335977            335971     392627              2   
146867    1158100   953845            953843      59396              1   
121958     591653     1905              1668    1173755              2   

        Match_Type  
297817           3  
128697           6  
67195            6  
36717            6  
146806

In [51]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
# Train the model
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

In [52]:
from sklearn.metrics import accuracy_score

# Calculate the accuracy
accuracy = accuracy_score(y_test, y_pred)

print("Accuracy: ", accuracy)

Accuracy:  0.5287126357226284
